In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

2024-01-13 20:15:54.741150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)
# 20231225 應該是當初安裝時程式有異，所以keras沒抓到tensorflow預設的版本
# 20240104 tf2.10 version can run
print(keras.__version__)

2.12.0
2.12.0


In [8]:
# Listing 6.1 Word-level one-hot encoding (toy example)
import numpy as np
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {} # 宣告一個字典變數
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

max_length = 10
"""
numpy zeros() :
shape：定義傳回陣列的形狀
dtype：產生矩陣的資料型，可選參數，預設為numpy.float64
order：{'C'，'F'}，可選，預設：'C'，是否在內容中以行（C）或列（F）順序儲存多維資料。
"""
results = np.zeros(shape=(len(samples), 
                          max_length, 
                          max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [15]:
print(results.shape)
print(results)
print(token_index)
#print(token_index.shape())
"""
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}
for sample in samples:
    for word in sample.split():
        print(word)
"""

(2, 10, 11)
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'mat.': 6, 'dog': 7, 'ate': 8, 'my': 9, 'homework.': 10}


"\nsamples = ['The cat sat on the mat.', 'The dog ate my homework.']\ntoken_index = {}\nfor sample in samples:\n    for word in sample.split():\n        print(word)\n"

In [59]:
# Listing 6.2 Character-level one-hot encoding (toy example)
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# string.printable
# value : 「0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$ ...」
# 由被視為可打印符號的ASCII字符組成的字符串。
# 這是digits, ascii_letters, punctuation 和 whitespace 的總和。
characters = string.printable
#print(characters)

#token_index = dict(zip(range(1, len(characters) + 1), characters))
token_index = dict(zip(characters, range(1, len(characters) + 1)))
#print(token_index)
#print(token_index.get('T'))

max_length = 50
print("== init results ==")
results = np.zeros((len(samples), max_length, len(token_index) + 1))
#print(results)

for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
#        print("character=", character, "index= ", index, "i= ", i, ",j= ", j)
        results[i, j, index] = 1.
print("== process results ==")
print(results.shape)
print(results)


== init results ==
== process results ==
(2, 50, 101)
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.

In [26]:
np.set_printoptions(threshold=np.inf)

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
#token_index = dict(zip(range(1, len(characters) + 1), characters))
print(list(enumerate(samples)))

#print(token_index)
#print(results.shape)
#print(results)


[(0, 'The cat sat on the mat.'), (1, 'The dog ate my homework.')]


In [54]:
# Listing 6.3 Using Keras for word-level one-hot encoding
# 自然語言處理的領域，tokenization 一般會翻譯做分詞，而 tokenizer 一般會翻譯成分詞器。
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index # 找回單詞索引

print('Found %s unique tokens.' % len(word_index))
print(word_index)


Found 9 unique tokens.
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


In [62]:
# Tokenizer 入門 ( https://ithelp.ithome.com.tw/articles/10298516 )

# Character tokenization
string = "Only those who will risk going too far can possibly find out how far one can go."
tokenized_str = list(string)
print(tokenized_str)

# numericalization
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_str)))}
print(token2idx)

# 把原始的句子，根據上面這個 set，轉換為數字
input_ids = [token2idx[token] for token in tokenized_str]
print(input_ids)


['O', 'n', 'l', 'y', ' ', 't', 'h', 'o', 's', 'e', ' ', 'w', 'h', 'o', ' ', 'w', 'i', 'l', 'l', ' ', 'r', 'i', 's', 'k', ' ', 'g', 'o', 'i', 'n', 'g', ' ', 't', 'o', 'o', ' ', 'f', 'a', 'r', ' ', 'c', 'a', 'n', ' ', 'p', 'o', 's', 's', 'i', 'b', 'l', 'y', ' ', 'f', 'i', 'n', 'd', ' ', 'o', 'u', 't', ' ', 'h', 'o', 'w', ' ', 'f', 'a', 'r', ' ', 'o', 'n', 'e', ' ', 'c', 'a', 'n', ' ', 'g', 'o', '.']
{' ': 0, '.': 1, 'O': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'k': 12, 'l': 13, 'n': 14, 'o': 15, 'p': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'w': 21, 'y': 22}
[2, 14, 13, 22, 0, 19, 10, 15, 18, 7, 0, 21, 10, 15, 0, 21, 11, 13, 13, 0, 17, 11, 18, 12, 0, 9, 15, 11, 14, 9, 0, 19, 15, 15, 0, 8, 3, 17, 0, 5, 3, 14, 0, 16, 15, 18, 18, 11, 4, 13, 22, 0, 8, 11, 14, 6, 0, 15, 20, 19, 0, 10, 15, 21, 0, 8, 3, 17, 0, 15, 14, 7, 0, 5, 3, 14, 0, 9, 15, 1]


In [67]:
# Word tokenization
string = "Only those who will risk going too far can possibly find out how far one can go."
tokenized_str = string.split()
print("tokenized_str= ", tokenized_str)

# numericalization
token_word2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_str)))} # 字典
print("token_word2idx= ", token_word2idx)

# mapping 回 set
input_ids = [token_word2idx[token] for token in tokenized_str]
print("input_ids= ", input_ids)

"""
enumerate() 函數用於將一個可遍歷的數據對象(如列表、元組或字符串)組合為一個索引序列，同時列出數據和數據下標，一般用在 for 循環當中。
syntax : enumerate(sequence, [start=0])
         sequence : 一個序列、迭代器或其他支持迭代對象。
         start    : 下標起始位置的值。

[set Memo]
Python set 集合初始化元素使用 {} 來包住元素，也可以帶入 set() 建構子，但若要建立空的 set 要使用 set()，
使用 s = {} 是會建立空 dict，不要搞錯囉！
Python 官方文件寫明 set 物件是無序，即使你印出來時發現是按照順序的，所以在使用 set 時請記得不保證有序的，
另外 set 裡是不會包含重複的元素。
"""

tokenized_str=  ['Only', 'those', 'who', 'will', 'risk', 'going', 'too', 'far', 'can', 'possibly', 'find', 'out', 'how', 'far', 'one', 'can', 'go.']
token_word2idx=  {'Only': 0, 'can': 1, 'far': 2, 'find': 3, 'go.': 4, 'going': 5, 'how': 6, 'one': 7, 'out': 8, 'possibly': 9, 'risk': 10, 'those': 11, 'too': 12, 'who': 13, 'will': 14}
input_ids=  [0, 11, 13, 14, 10, 5, 12, 2, 1, 9, 3, 8, 6, 2, 7, 1, 4]


In [72]:
# Listing 6.4 Word-level one-hot encoding with hashing trick (toy example)
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
dimensionality = 1000
max_length = 10
results = np.zeros((len(samples), max_length, dimensionality))

print(list(enumerate(samples)))
print(list(enumerate(sample.split())))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

#print(results.shape)
print(results)

[(0, 'The cat sat on the mat.'), (1, 'The dog ate my homework.')]
[(0, 'The'), (1, 'dog'), (2, 'ate'), (3, 'my'), (4, 'homework.')]
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0

In [ ]:
# Listing 6.5 Instantiating an Embedding layer
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)
